# Step 3: Multiple Imputation by Chained Equations (MICE)

In [1]:
from ml_tools.MICE_imputation import run_mice_pipeline
from paths import DATA_DIR, ENGINEERED_CSVS_DIR, MICE_IMPUTED_DATASETS_DIR, MICE_METRICS_DIR
from helpers.constants import TARGETS
from ml_tools.utilities import deserialize_object
import os

In [2]:
binary_columns_path = os.path.join(DATA_DIR, "BINARY_COLUMNS_list.joblib")
binary_columns = deserialize_object(filepath=binary_columns_path)

✅ Loaded object of type '<class 'list'>'


In [6]:
run_mice_pipeline(df_path_or_dir=ENGINEERED_CSVS_DIR,
                  target_columns=TARGETS,
                  save_datasets_dir=MICE_IMPUTED_DATASETS_DIR,
                  save_metrics_dir=MICE_METRICS_DIR,
                  binary_columns=binary_columns,
                  iterations=30)

🗂️ CSV files found:
	engineered_dataset

💿 Loaded dataset: 'engineered_dataset' with shape: (1220, 119)
✅ Saved dataset: 'engineered_dataset_MICE.csv' with shape: (1220, 119)
Convergence_Metrics_engineered_dataset_MICE completed.
Distribution_Metrics_engineered_dataset_imputed completed.
